In [1]:
from interpreter import interpreter
import os

In [4]:
interpreter.llm.model = "azure/gpt-4.1"
interpreter.llm.api_key = os.getenv("AZURE_OPENAI_API_KEY")
interpreter.llm.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
interpreter.llm.api_version = os.getenv("AZURE_OPENAI_API_VERSION")
interpreter.auto_run = True
interpreter.system_message = "You are a helpful assistant, who use maths to answer questions."

output = interpreter.chat("who are you?")

print(output)

[{'role': 'assistant', 'type': 'message', 'content': 'I am a helpful AI assistant designed to answer your questions, solve problems using mathematics, and provide guidance and support. Let me know how I can assist you today!'}]


In [9]:
message_with_image = [
        {"role": "user", "type": "message", "content": "Describe this image and then, using pyautogui, press ctrl + t to open a new tab in the browser."},
        {"role": "user", "type": "image", "format": "path", "content": "image.png"}
    ]
output = interpreter.chat(message_with_image)
print(output)

c:\Users\sathish\miniconda3\envs\py3.10\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[{'role': 'assistant', 'type': 'message', 'content': "Description of the image:\nThe image shows a road disappearing into the horizon with an overpass crossing above it. The sky is cloudy with hints of color from the setting (or rising) sun. In the foreground, a person's hand is holding a mobile phone, using it to take a picture of the same scene. The phone's screen displays the road and the overpass, mimicking the main view in the photo.\n\nNext, to open a new tab in your browser using pyautogui, I will execute the keyboard shortcut ctrl + t."}, {'role': 'assistant', 'type': 'code', 'format': 'python', 'content': "import pyautogui\npyautogui.hotkey('ctrl', 't')"}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': ''}, {'role': 'assistant', 'type': 'message', 'content': 'I have described the image and executed the pyautogui command to press Ctrl + T, which should open a new tab in your browser. \n\nIf you need further automation or have another task, please let me 

[{'role': 'assistant', 'type': 'message', 'content': "Description of the image:\nThe image shows a road disappearing into the horizon with an overpass crossing above it. The sky is cloudy with hints of color from the setting (or rising) sun. In the foreground, a person's hand is holding a mobile phone, using it to take a picture of the same scene. The phone's screen displays the road and the overpass, mimicking the main view in the photo.\n\nNext, to open a new tab in your browser using pyautogui, I will execute the keyboard shortcut ctrl + t."}, {'role': 'assistant', 'type': 'code', 'format': 'python', 'content': "import pyautogui\npyautogui.hotkey('ctrl', 't')"}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': ''}, {'role': 'assistant', 'type': 'message', 'content': 'I have described the image and executed the pyautogui command to press Ctrl + T, which should open a new tab in your browser. \n\nIf you need further automation or have another task, please let me know!'}]

In [ ]:
[{'role': 'assistant', 'type': 'message', 'content': "Description of the image:\nThe image shows a road disappearing into the horizon with an overpass crossing above it. The sky is cloudy with hints of color from the setting (or rising) sun. In the foreground, a person's hand is holding a mobile phone, using it to take a picture of the same scene. The phone's screen displays the road and the overpass, mimicking the main view in the photo.\n\nNext, to open a new tab in your browser using pyautogui, I will execute the keyboard shortcut ctrl + t."}, {'role': 'assistant', 'type': 'code', 'format': 'python', 'content': "import pyautogui\npyautogui.hotkey('ctrl', 't')"}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': ''}, {'role': 'assistant', 'type': 'message', 'content': 'I have described the image and executed the pyautogui command to press Ctrl + T, which should open a new tab in your browser. \n\nIf you need further automation or have another task, please let me know!'}]